In [178]:
import pandas as pd
import numpy as np
from Ames_Functions import * #OSMR, get_geoapify, get_dist, locator, find_min_dist, load_state_pkl, save_state_pkl
import pickle
pd.set_option("display.max_columns", None)

## House Latlongs Lookup Section

In [36]:
df_RE_latest = pd.read_csv("data/Ames_Real_Estate_Data_Latest.csv")
df_EssentialLatLong = pd.read_csv("df_EssentialLatLong_v2.csv")

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_12192/1833454112.py:1: DtypeWarning: Columns (36,40,81,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df_RE_latest = pd.read_csv("data/Ames_Real_Estate_Data_Latest.csv")


In [51]:
df_RE_latlongs.iloc[97,:].to_frame().T

,MapRefNo,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,...,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,reverse_lookup_rd
1354,527359170.0,527359170,0,0,2989 MONROE DR,RL,NaN,NaN,NaN,RESIDENTIAL,...,NaN,MONROE,DR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,4.0,NaN


In [ ]:
#Storing Latlongs using Geoapify

df_RE_latlongs = df_RE_latest.query('YrSold_YYYY == 2019').drop_duplicates()
latlongs = df_RE_latlongs.Prop_Addr.apply(lambda x: get_geoapify(x + ", Ames, Iowa"))

In [ ]:
# #Storing Latlongs using Geoapify

# df_RE_latlongs = df_RE_latest.query('YrSold_YYYY == 2021').drop_duplicates()
# latlongs = df_RE_latlongs.Prop_Addr.apply(lambda x: get_geoapify(x + ", Ames, Iowa"))

In [8]:
df_RE_latest.query('YrSold_YYYY == 2019').duplicated().sum()

3

In [ ]:
#Checking Latlongs with reverse geocoding
#latlongs.tail(2)
s_address = latlongs.apply(lambda x: locator.reverse(x).raw['address'])
s_address

In [78]:
#pd.json_normalize(s_address).join(pd.Series(s_address.index, name = 'RE_index')).query("index == 96 or index == 97")
s_road = pd.json_normalize(s_address).road.fillna("Monroe Drive")
s_road.to_frame()

,road
0,Mcfarland Avenue
1,Mcfarland Avenue
2,Ledges Drive
3,Ledges Drive
4,Ledges Drive
...,...
896,Ken Maril Road
897,Ken Maril Road
898,Ken Maril Road
899,Ken Maril Road


In [135]:
df_RE_latlongs = df_RE_latlongs.join(latlongs.str.split(",", expand = True).set_axis(["Lat", "Long"], axis = 1))

In [138]:
#df_RE_latlongs[["PA-Strt"]].str.lower()
df_RE_latlongs['reverse_lookup_rd'] = s_road.values
mask = df_RE_latlongs.apply(lambda x: x["PA-Strt"].lower() not in x.reverse_lookup_rd.lower(), axis = 1)
df_RE_latlongs.query('@mask')[["Prop_Addr", "reverse_lookup_rd", "Lat", "Long"]].to_csv("df_RE_latlongs2019_cleanup.csv")
#df_RE_latlongs["PA-Strt"].str.lower()

In [147]:
df_RE_latlongs.loc[df_latlong_corrections.index, ["Lat", "Long"]] = df_latlong_corrections[["Lat", "Long"]]
df_RE_latlongs.loc[df_latlong_corrections.index]

,MapRefNo,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,...,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,reverse_lookup_rd,Lat,Long
11371,906329170.0,906329170,0,0,5116 SPRINGBROOK CIR,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,CIR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,2.0,6th Street,42.028568,-93.687674
11373,906329180.0,906329180,0,0,5122 SPRINGBROOK CIR,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,CIR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,2.0,6th Street,42.028348,-93.687718


In [144]:
df_latlong_corrections = pd.read_csv("df_RE_latlongs2019_corrections.csv", index_col = 0)
df_latlong_corrections

,Prop_Addr,reverse_lookup_rd,Lat,Long
11371,5116 SPRINGBROOK CIR,6th Street,42.028568,-93.687674
11373,5122 SPRINGBROOK CIR,6th Street,42.028348,-93.687718


In [150]:
Ames_notebook_state = {}
Ames_notebook_state["df_RE_latlong2019"] = (df_RE_latlongs, "latlong lookup for 2019 using geoapify and reverse lookup using nominatim with corrections done by hand")

In [152]:
pickle.dump(Ames_notebook_state, open("Ames_notebook_state.pkl", "wb"))

In [3]:
Ames_notebook_state = load_state_pkl()

In [13]:
df_RE_latlongs = Ames_notebook_state['df_RE_latlong2019'][0]

In [16]:
#This should be added into the function find_min_dist

# df_RE_latlongs.Lat = df_RE_latlongs.Lat.astype(float)
# df_RE_latlongs.Long = df_RE_latlongs.Long.astype(float)
# df_RE_latlongs.index.name = 'SaleID'
# df_RE_latlongs.reset_index(inplace = True)

In [179]:
df_min_dist.filter(regex = "_dist$").join(df_min_dist.SalePrice).describe()

Service,SaleID,Prop_Addr,SalePrice,Arts_dist,Elem_School_dist,Golf_dist,Gym_dist,High_School_dist,Historic_dist,ISU_dist,Library_dist,Medical_dist,Movies_dist,Organic_Groceries_dist,Park_dist,Recreation_dist,Religion_dist,Restaurant_dist,Shopping_dist,Spa_dist,Sports_Venue_dist,Arts_name,Elem_School_name,Golf_name,Gym_name,High_School_name,Historic_name,ISU_name,Library_name,Medical_name,Movies_name,Organic_Groceries_name,Park_name,Recreation_name,Religion_name,Restaurant_name,Shopping_name,Spa_name,Sports_Venue_name
0,92,1904 KETELSEN DR,125000.0,3.621211,1.753512,1.169253,1.899291,2.501688,3.354657,3.284050,3.570212,3.207433,4.464579,1.114132,0.643165,2.238390,1.246691,1.837750,1.839250,1.815357,4.207073,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic District,Iowa State University,Ames Public Library,Mary Greeley Medical Center,Cinemark Movies 12,Fareway Grocery,Ada Hayden Heritage Park,Inis Grove Park/Rec,Christ Community Church,The Cafe,North Grand Mall/Walmart SuperCenter B,Serenity Couture Salon and Spa,Jack Trice Stadium
1,93,1910 KETELSEN DR,100000.0,3.621211,1.753512,1.169253,1.899291,2.501688,3.354657,3.284050,3.570212,3.207433,4.464579,1.114132,0.643165,2.238390,1.246691,1.837750,1.839250,1.815357,4.207073,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic District,Iowa State University,Ames Public Library,Mary Greeley Medical Center,Cinemark Movies 12,Fareway Grocery,Ada Hayden Heritage Park,Inis Grove Park/Rec,Christ Community Church,The Cafe,North Grand Mall/Walmart SuperCenter B,Serenity Couture Salon and Spa,Jack Trice Stadium
2,96,1407 LEDGES DR,399000.0,3.710401,1.908894,1.462515,2.147550,2.679027,3.445747,3.506823,3.658191,3.276203,4.568679,1.402202,0.465953,2.269335,1.549200,2.085425,1.915365,1.898055,4.393030,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic District,Iowa State University,Ames Public Library,Mary Greeley Medical Center,Cinemark Movies 12,Fareway Grocery,Ada Hayden Heritage Park,Inis Grove Park/Rec,Christ Community Church,The Cafe,North Grand Mall/Walmart SuperCenter B,Serenity Couture Salon and Spa,Jack Trice Stadium
3,98,1415 LEDGES DR,36000.0,3.710715,1.908000,1.457711,2.145128,2.677882,3.446017,3.504953,3.658521,3.276795,4.568804,1.398923,0.468996,2.270374,1.544397,2.083007,1.915682,1.898271,4.391834,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic District,Iowa State University,Ames Public Library,Mary Greeley Medical Center,Cinemark Movies 12,Fareway Grocery,Ada Hayden Heritage Park,Inis Grove Park/Rec,Christ Community Church,The Cafe,North Grand Mall/Walmart SuperCenter B,Serenity Couture Salon and Spa,Jack Trice Stadium
4,100,1503 LEDGES DR,36000.0,3.720136,1.904619,1.405491,2.124316,2.671482,3.454992,3.490261,3.668111,3.289235,4.576120,1.368188,0.507822,2.287727,1.492540,2.062262,1.925499,1.907006,4.384658,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic District,Iowa State University,Ames Public Library,Mary Greeley Medical Center,Cinemark Movies 12,Fareway Grocery,Ada Hayden Heritage Park,Inis Grove Park/Rec,Christ Community Church,The Cafe,North Grand Mall/Walmart SuperCenter B,Serenity Couture Salon and Spa,Jack Trice Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,22745,500 KEN MARIL RD,175000.0,2.719105,1.797315,1.914441,1.734158,3.894992,2.810964,3.522879,2.772280,3.179553,1.852297,2.821734,2.440837,2.838125,2.543357,1.893270,2.389929,2.663162,2.487271,Octagon Center For the Art

## Driving time Lookups Section

In [25]:
df_cross = df_RE_latlongs.join(df_EssentialLatLong, how = "cross", lsuffix = '_house', rsuffix = "_biz")
df_cross["dist"] = df_cross.apply(lambda x: get_dist(np.array([x.Lat_house, x.Long_house]), np.array([x.Lat_biz, x.Long_biz])), axis = 1)
df_subset = df_cross.loc[df_cross.groupby(["Service", "SaleID"]).dist.idxmin()]

In [ ]:
df_subset

In [32]:
# df_cross = pd.read_csv("df_cross.csv", index_col = 0)
# df_mins = df_cross.loc[df_cross.groupby(["Service", "SaleID"]).dist.idxmin()] #THIS IS EQUIVALENT TO df_subset

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_80585/2705128043.py:1: DtypeWarning: Columns (9,10,35,36,39,40,84,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cross = pd.read_csv("df_cross.csv", index_col = 0)


In [ ]:
# #Below code is from geoapify website

# import requests

# # Replace YOUR_API_KEY with your actual API key. Sign up and get an API key on https://www.geoapify.com/ 
# API_KEY = "c65731ef1fc94fd3aba9a53df17c2366"

# # Define the address to geocode
# address = "1505 LITTLE BLUESTEM CT UNIT 101, Ames, Iowa"

# def get_geoapify (address):
#     # Build the API URL
#     url = f"https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={API_KEY}"

#     # Send the API request and get the response
#     response = requests.get(url)

#     # Check the response status code
#     if response.status_code == 200:
#         # Parse the JSON data from the response
#         data = response.json()

#         # Extract the first result from the data
#         result = data["features"][0]

#         # Extract the latitude and longitude of the result
#         latitude = result["geometry"]["coordinates"][1]
#         longitude = result["geometry"]["coordinates"][0]

#         print(f"Latitude: {latitude}, Longitude: {longitude}")
#     else:
#         print(f"Request failed with status code {response.status_code}")
#     return f"{latitude}, {longitude}"

In [57]:
#from_biz = {}
OSMR_list = df_subset.Service.unique()

for i in OSMR_list[11:]:
    from_biz[i] = df_subset.query(f"Service == '{i}'").apply(lambda x: OSMR(x.Lat_biz, x.Long_biz, x.Lat_house, x.Long_house), axis = 1)
    
pd.DataFrame(from_biz)

,Arts,Elem_School,Golf,Gym,High_School,Historic,ISU,Library,Medical,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Spa,Sports_Venue
2,"[[642.8, 8106.8]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"[[496.8, 4749.3]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,"[[396.8, 2910.4]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,"[[498.0, 4405.1]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,"[[654.6, 7138.7]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[63.5, 717.2]]",NaN,NaN
36933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[153.8, 1290.8]]",NaN
36935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[416.9, 3971.8]]"
36937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[166.2, 1532.4]]",NaN,NaN,NaN


In [39]:
from_biz_test = df_subset.head(3).query("Service == 'Arts'").apply(lambda x: OSMR(x.Lat_biz, x.Long_biz, x.Lat_house, x.Long_house), axis = 1)
from_biz_test

2     [[642.8, 8106.8]]
43    [[642.8, 8106.8]]
84    [[539.9, 7384.7]]
dtype: object

In [53]:
OSMR_list[6:11]

array(['ISU', 'Library', 'Medical', 'Movies', 'Organic_Groceries'],
      dtype=object)

In [58]:
#from_biz["Arts"] = df_subset.query("Service == 'Arts'").apply(lambda x: OSMR(x.Lat_biz, x.Long_biz, x.Lat_house, x.Long_house), axis = 1)
save_state_pkl('from_biz_api', from_biz, "Adding first 6 services driving times/distances lookups from OSMR")

In [85]:
from_biz_final2019 = pd.DataFrame(load_state_pkl()["from_biz_api"][0])
from_biz_final2019

,Arts,Elem_School,Golf,Gym,High_School,Historic,ISU,Library,Medical,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Spa,Sports_Venue
2,"[[642.8, 8106.8]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"[[496.8, 4749.3]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,"[[396.8, 2910.4]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,"[[498.0, 4405.1]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,"[[654.6, 7138.7]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[63.5, 717.2]]",NaN,NaN
36933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[153.8, 1290.8]]",NaN
36935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[416.9, 3971.8]]"
36937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[166.2, 1532.4]]",NaN,NaN,NaN


In [77]:
from_biz_final = pickle.load(open("../ML_Project_Sandbox/from_biz_final.pkl", 'rb'))
from_biz_final

,Golf,Gym,Arts,Elem_School,Spa,High_School,Historic,ISU,Library,Medical,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Sports_Venue
1,NaN,NaN,"[[633.1, 6840.0]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"[[340.3, 4065.6]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"[[131.2, 1489.0]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,"[[288.2, 3129.1]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,"[[500.7, 5288.8]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[273.6, 4091.9]]",NaN
25945,NaN,NaN,NaN,NaN,"[[363.9, 4665.5]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[627.0, 7346.6]]"
25949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[376.3, 4907.1]]",NaN,NaN


In [36]:
from_biz_stacked = from_biz_final.stack().reset_index().set_axis(["df_cross_index", "Service", "time_dist"], axis = 1)

from_biz_temp = from_biz_stacked.join(from_biz_stacked
                                            .apply(lambda x: f"{x.time_dist[0][0]}, {x.time_dist[0][1]}", axis = 1)
                                            .str.split(", ", expand = True)
                                            .set_axis(["time_secs", "driving_dist_meters"], axis = 1)
                                            .apply(lambda x: x.astype(float))
                                    ).drop("time_dist", axis = 1)

In [37]:
df_driv_final = {}

for k, f in zip(["from_time", "from_driv", "dist"], ["time_secs", "driving_dist_meters", "dist"]):
        df_driv_final[k] = (df_mins
                                .join(from_biz_temp.set_index("df_cross_index"), rsuffix = "_driv")
                                .query("Service == Service_driv")
                                .drop("Service_driv", axis = 1)
                                .pivot(index = ["SaleID", "Lat_house", "Long_house"], columns = "Service", values = f)
                                .reset_index()
                                .set_index("SaleID").join(df_mins[["SaleID", "SalePrice"]].drop_duplicates().set_index("SaleID").SalePrice)
                        )
        
df_driv_final["from_driv"] #.set_index("SaleID").join(df_mins[["SaleID", "SalePrice"]].drop_duplicates().set_index("SaleID").SalePrice)

,Lat_house,Long_house,Arts,Elem_School,Golf,Gym,High_School,Historic,ISU,Library,...,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Spa,Sports_Venue,SalePrice
SaleID,,,,,,,,,,,,,,,,,,,,,
2,42.064191,-93.662310,6840.0,4065.6,1489.0,3129.1,5288.8,7728.4,5622.6,6682.4,...,9956.7,3010.9,6083.0,5578.4,1184.7,3112.5,5076.9,4722.2,7714.0,78000.0
3,42.064191,-93.662310,6840.0,4065.6,1489.0,3129.1,5288.8,7728.4,5622.6,6682.4,...,9956.7,3010.9,6083.0,5578.4,1184.7,3112.5,5076.9,4722.2,7714.0,408905.0
18,42.063979,-93.662658,6860.7,4086.2,1509.7,3149.8,5309.5,7749.1,5643.3,6703.0,...,9977.4,3031.6,6103.7,5599.1,1205.3,3133.2,5097.6,4742.8,7734.6,434066.0
19,42.063784,-93.662604,6882.6,4108.2,1531.6,3171.8,5331.5,7771.1,5665.3,6725.0,...,9999.4,3053.5,6125.7,5621.1,1227.3,3155.2,5119.5,4764.8,7756.6,88000.0
20,42.063575,-93.662609,6905.8,4131.3,1554.7,3194.9,5354.6,7794.2,5688.4,6748.1,...,10022.5,3076.7,6148.8,5644.2,1250.4,3178.3,5142.6,4787.9,7779.7,88000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,41.986503,-93.606026,4801.4,3859.6,3721.5,5334.0,7854.0,5458.3,8031.8,4886.9,...,3366.1,6029.1,5213.3,5415.7,7948.9,4099.4,4165.9,5221.6,5512.4,220000.0
22749,41.986496,-93.603619,5001.0,4059.2,3921.1,5533.6,8053.6,5657.9,8231.4,5086.5,...,3565.7,6228.7,5412.9,5615.3,8148.5,4299.0,4365.4,5421.2,5712.0,217500.0
22756,41.986499,-93.602049,5131.0,4189.3,4051.1,5663.7,8183.6,5788.0,8361.4,5216.6,...,3695.7,6358.7,5543.0,5745.4,8278.6,4429.1,4495.5,5551.2,5842.0,229000.0


In [ ]:
pickle.dump(df_driv_final, open("df_driv_final.pkl", 'wb'))

In [88]:
df_subset.sort_index()

,SaleID,MapRefNo,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,...,reverse_lookup_rd,Lat_house,Long_house,Name,Address,Service,Lat_biz,Long_biz,Label,dist
2,92,5.221040e+08,522104010,0,0,1904 KETELSEN DR,FS-RL,NaN,NaN,NaN,...,Mcfarland Avenue,42.074875,-93.635298,Octagon Center For the Arts,"427 Douglas Ave, Ames, IA 50010",Arts,42.025416,-93.612190,Oct_Art,3.621211
3,92,5.221040e+08,522104010,0,0,1904 KETELSEN DR,FS-RL,NaN,NaN,NaN,...,Mcfarland Avenue,42.074875,-93.635298,St. Cecilia Elementary School,"2900 HOOVER AVE, AMES, IA 50010",Elem_School,42.049819,-93.630113,St_C_School,1.753512
7,92,5.221040e+08,522104010,0,0,1904 KETELSEN DR,FS-RL,NaN,NaN,NaN,...,Mcfarland Avenue,42.074875,-93.635298,Ames Golf & Country Club,"5752 George Washington Carver Ave, Ames, IA 50010",Golf,42.074036,-93.658043,Ames_GC,1.169253
10,92,5.221040e+08,522104010,0,0,1904 KETELSEN DR,FS-RL,NaN,NaN,NaN,...,Mcfarland Avenue,42.074875,-93.635298,Ames Fitness Center - North,"2622 Stange Rd #101, Ames, IA 50010",Gym,42.048206,-93.644097,Fitness_North,1.899291
12,92,5.221040e+08,522104010,0,0,1904 KETELSEN DR,FS-RL,NaN,NaN,NaN,...,Mcfarland Avenue,42.074875,-93.635298,Ames High School,"1801 Ridgewood Ave, Ames, IA 50010",High_School,42.038750,-93.632958,Ames_HS,2.501688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36931,22797,1.004300e+09,1004300375,0,0,5501 E LINCOLN WAY,A,NaN,NaN,NaN,...,East Lincoln Way,42.022834,-93.604921,Walmart SuperCenter A,"534 S Duff Ave, Ames, IA 50010",Shopping,42.020810,-93.610204,Walmart_A,0.305431
36933,22797,1.004300e+09,1004300375,0,0,5501 E LINCOLN WAY,A,NaN,NaN,NaN,...,East Lincoln Way,42.022834,-93.604921,Massage Heights Ames,"637 Lincoln Way #103, Ames, IA 50010",Spa,42.023374,-93.619612,Massage_Heights_Spa,0.755812
36935,22797,1.004300e+09,1004300375,0,0,5501 E LINCOLN WAY,A,NaN,NaN,NaN,...,East Lincoln Way,42.022834,-93.604921,Jack Trice Stadium,"1732 S 4th St, Ames, IA 50011",Sports_Venue,42.014055,-93.635788,JT_Stadium,1.698486
36937,22797,1.004300e+09,1004300375,0,0,5501 E LINCOLN WAY,A,NaN,NaN,NaN,...,East Lincoln Way,42.022834,-93.604921,Aunt Maude's,"547 Main St, Ames, IA 50010",Restaurant,42.025323,-93.619079,Aunt_Maude_Resto,0.747562


## Creating 2019 dfs after API drive time/distance lookups in preparation for feature engineering 

In [96]:
from_biz_stacked2019 = from_biz_final2019.stack().reset_index().set_axis(["df_subset_index", "Service", "time_dist"], axis = 1)

from_biz_temp2019 = from_biz_stacked2019.join(from_biz_stacked2019
                                            .apply(lambda x: f"{x.time_dist[0][0]}, {x.time_dist[0][1]}", axis = 1)
                                            .str.split(", ", expand = True)
                                            .set_axis(["time_secs", "driving_dist_meters"], axis = 1)
                                            .apply(lambda x: x.astype(float))
                                            ).drop("time_dist", axis = 1)

In [98]:
driv_final_dict2019 = {}

for k, f in zip(["from_time", "from_driv", "dist"], ["time_secs", "driving_dist_meters", "dist"]):
        driv_final_dict2019[k] = (df_subset
                                .join(from_biz_temp2019.set_index("df_subset_index"), rsuffix = "_driv")
                                .query("Service == Service_driv")
                                .drop("Service_driv", axis = 1)
                                .pivot(index = ["SaleID", "Lat_house", "Long_house"], columns = "Service", values = f)
                                .reset_index()
                                .set_index("SaleID").join(df_subset[["SaleID", "SalePrice"]].drop_duplicates().set_index("SaleID").SalePrice)
                        )

driv_final_dict2019["from_driv"] #.set_index("SaleID").join(df_mins[["SaleID", "SalePrice"]].drop_duplicates().set_index("SaleID").SalePrice)

,Lat_house,Long_house,Arts,Elem_School,Golf,Gym,High_School,Historic,ISU,Library,...,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Spa,Sports_Venue,SalePrice
SaleID,,,,,,,,,,,,,,,,,,,,,
92,42.074875,-93.635298,8106.8,4749.3,2910.4,4405.1,7138.7,7562.5,7010.2,8020.9,...,9790.8,3015.9,2623.5,5499.3,2930.6,4388.5,4911.0,4768.5,10451.4,125000.0
93,42.074875,-93.635298,8106.8,4749.3,2910.4,4405.1,7138.7,7562.5,7010.2,8020.9,...,9790.8,3015.9,2623.5,5499.3,2930.6,4388.5,4911.0,4768.5,10451.4,100000.0
96,42.077416,-93.629924,7384.7,4786.2,3118.4,4613.1,6416.6,6840.4,8948.1,7298.8,...,9068.7,3224.0,1901.4,4777.2,3138.6,4596.6,4188.9,4046.3,9729.2,399000.0
98,42.077403,-93.630016,7392.4,4793.9,3126.2,4620.9,6424.3,6848.1,8955.8,7306.5,...,9076.4,3231.7,1909.1,4784.9,3146.4,4604.3,4196.6,4054.1,9736.9,36000.0
100,42.077346,-93.631034,7478.7,4880.2,3212.5,4707.2,6510.6,6934.4,9042.1,7392.8,...,9162.7,3318.0,1995.4,4871.2,3232.7,4690.6,4282.9,4140.4,9823.2,36000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22745,41.986502,-93.604697,4911.6,3969.9,3831.7,5444.3,7964.2,5568.6,8142.1,4997.2,...,3476.3,6139.3,5323.6,5526.0,8059.2,4209.7,4276.1,5331.8,5622.6,175000.0
22747,41.986500,-93.604611,4918.8,3977.0,3838.9,5451.4,7971.3,5575.7,8149.2,5004.3,...,3483.4,6146.4,5330.7,5533.1,8066.3,4216.8,4283.2,5338.9,5629.7,137500.0
22750,41.986497,-93.603528,5008.5,4066.8,3928.6,5541.2,8061.1,5665.5,8238.9,5094.1,...,3573.2,6236.2,5420.5,5622.9,8156.0,4306.6,4373.0,5428.7,5719.5,200000.0


In [184]:
df_min_time = (df_subset.join(from_biz_temp2019.set_index("df_subset_index"), rsuffix = "_driv")
            .drop("Service_driv", axis = 1)
            .pivot(index = ["SaleID"], columns = "Service", values = "time_secs")
            .reset_index()
            .set_index("SaleID")
            .add_suffix("_time")
            #.join(df_subset[["SaleID"]].drop_duplicates().set_index("SaleID"))
            )

In [ ]:
# df_min_dist = (df_subset
#             .pivot(index = ["SaleID"], columns = "Service", values = "dist")
#             .reset_index()
#             .set_index("SaleID")
#             )

In [196]:
df_min_dist = find_min_dist(df_RE_latlongs, df_EssentialLatLong).set_index("SaleID")

In [191]:
def service_pivot (left_df, right_df, v):
    return left_df.copy().join(right_df.pivot(columns = "Service", index = "SaleID", values = v).add_suffix("_" + v))


#(df_subset.pivot(columns = "Service", index = "SaleID", values = "Name"))#.filter(regex = "Lat_|Long_|Prop_Addr|Name|Address")

In [212]:
save_state_pkl("2019dfs_for_feature_engineering",
               df_min_dist.join(df_min_time).pipe(service_pivot, df_subset, "Lat_biz").pipe(service_pivot, df_subset, "Long_biz").drop("SalePrice", axis = 1).join(df_min_dist.SalePrice),
               "After data transformation of the driving time/distance API results in preparation for feature engineering. Also includes business information and point to point distances."
                )

In [115]:
save_state_pkl("2019dfs_for_feature_engineering", driv_final_dict2019, "After data transformation of the driving time/distance API results in preparation for feature engineering")

In [205]:
save_state_pkl("2019dfs_for_feature_engineering", None, "Removing dictionary version of 2019dfs for feature engineering and replacing with master dataframe with all the same information")

In [216]:
load_state_pkl()["2019dfs_for_feature_engineering"][0]

,Prop_Addr,Arts_dist,Elem_School_dist,Golf_dist,Gym_dist,High_School_dist,Historic_dist,ISU_dist,Library_dist,Medical_dist,Movies_dist,Organic_Groceries_dist,Park_dist,Recreation_dist,Religion_dist,Restaurant_dist,Shopping_dist,Spa_dist,Sports_Venue_dist,Arts_name,Elem_School_name,Golf_name,Gym_name,High_School_name,Historic_name,ISU_name,Library_name,Medical_name,Movies_name,Organic_Groceries_name,Park_name,Recreation_name,Religion_name,Restaurant_name,Shopping_name,Spa_name,Sports_Venue_name,Arts_time,Elem_School_time,Golf_time,Gym_time,High_School_time,Historic_time,ISU_time,Library_time,Medical_time,Movies_time,Organic_Groceries_time,Park_time,Recreation_time,Religion_time,Restaurant_time,Shopping_time,Spa_time,Sports_Venue_time,Arts_Lat_biz,Elem_School_Lat_biz,Golf_Lat_biz,Gym_Lat_biz,High_School_Lat_biz,Historic_Lat_biz,ISU_Lat_biz,Library_Lat_biz,Medical_Lat_biz,Movies_Lat_biz,Organic_Groceries_Lat_biz,Park_Lat_biz,Recreation_Lat_biz,Religion_Lat_biz,Restaurant_Lat_biz,Shopping_Lat_biz,Spa_Lat_biz,Sports_Venue_Lat_biz,Arts_Long_biz,Elem_School_Long_biz,Golf_Long_biz,Gym_Long_biz,High_School_Long_biz,Historic_Long_biz,ISU_Long_biz,Library_Long_biz,Medical_Long_biz,Movies_Long_biz,Organic_Groceries_Long_biz,Park_Long_biz,Recreation_Long_biz,Religion_Long_biz,Restaurant_Long_biz,Shopping_Long_biz,Spa_Long_biz,Sports_Venue_Long_biz,SalePrice
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,1904 KETELSEN DR,3.621211,1.753512,1.169253,1.899291,2.501688,3.354657,3.284050,3.570212,3.207433,4.464579,1.114132,0.643165,2.238390,1.246691,1.837750,1.839250,1.815357,4.207073,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic District,Iowa State University,Ames Public Library,Mary Greeley Medical Center,Cinemark Movies 12,Fareway Grocery,Ada Hayden Heritage Park,Inis Grove Park/Rec,Christ Community Church,The Cafe,North Grand Mall/Walmart SuperCenter B,Serenity Couture Salon and Spa,Jack Trice Stadium,642.8,496.8,396.8,498.0,654.6,626.9,753.3,630.3,585.7,798.3,352.7,346.0,484.2,370.8,488.4,442.1,451.0,902.9,42.025416,42.049819,42.074036,42.048206,42.03875,42.029014,42.027905,42.026189,42.032251,42.012801,42.060334,42.073046,42.047198,42.072546,42.049041,42.050573,42.050679,42.014055,-93.61219,-93.630113,-93.658043,-93.644097,-93.632958,-93.614057,-93.644613,-93.612219,-93.61071,-93.611484,-93.644629,-93.623016,-93.612713,-93.659376,-93.643649,-93.620766,-93.621608,-93.635788,125000.0
93,1910 KETELSEN DR,3.621211,1.753512,1.169253,1.899291,2.501688,3.354657,3.284050,3.570212,3.207433,4.464579,1.114132,0.643165,2.238390,1.246691,1.837750,1.839250,1.815357,4.207073,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic District,Iowa State University,Ames Public Library,Mary Greeley Medical Center,Cinemark Movies 12,Fareway Grocery,Ada Hayden Heritage Park,Inis Grove Park/Rec,Christ Community Church,The Cafe,North Grand Mall/Walmart SuperCenter B,Serenity Couture Salon and Spa,Jack Trice Stadium,642.8,496.8,396.8,498.0,654.6,626.9,753.3,630.3,585.7,798.3,352.7,346.0,484.2,370.8,488.4,442.1,451.0,902.9,42.025416,42.049819,42.074036,42.048206,42.03875,42.029014,42.027905,42.026189,42.032251,42.012801,42.060334,42.073046,42.047198,42.072546,42.049041,42.050573,42.050679,42.014055,-93.61219,-93.630113,-93.658043,-93.644097,-93.632958,-93.614057,-93.644613,-93.612219,-93.61071,-93.611484,-93.644629,-93.623016,-93.612713,-93.659376,-93.643649,-93.620766,-93.621608,-93.635788,100000.0
96,1407 LEDGES DR,3.710401,1.908894,1.462515,2.147550,2.679027,3.445747,3.506823,3.658191,3.276203,4.568679,1.402202,0.465953,2.269335,1.549200,2.085425,1.915365,1.898055,4.393030,Octagon Center For the Arts,St. Cecilia Elementary School,Ames Golf & Country Club,Ames Fitness Center - North,Ames High School,Old Town Historic Distri